# 7. Advanced Deep Learning Best Practices

### Callbacks

When training a model, there are many things you cannot predict from the start. For example you cannot predict the number of epochs needed to get an optimal validation loss. Typically, you train for the specified number of epochs, then select the optimal number of epochs to train for and train again. This approach is wasteful.

A much better way to handle this is to stop training when you realise the validation loss is no longer improving. This can be achieved using a Keras callback. A <b>callback</b> is an object that is passed in the model in the call to `fit()` and is called by the model at various points during training. It has access to all the availebl data about the state of the model and can take various actions: interrupt training, save model, load weight sets etc. Some common callbacks are:

- `callbacks.ModelCheckpoint`
- `callbacks.EarlyStopping`
- `callbacks.LearningRateScheduler`
- `callbacks.ReduceLROnPlateau`
- `callbacks.CSVLogger`

In [ ]:
input_layer = tf_keras.layers.Input(shape=NUM_FEATURES)
hidden_layer1 = tf_keras.layers.Dense(30, activation='relu')(input_layer)
hidden_layer2 = tf_keras.layers.Dense(30, activation='relu')(hidden_layer1)
concat_layer = tf_keras.layers.Concatenate()([input_layer, hidden_layer2])
output_layer = tf_keras.layers.Dense(1)(concat_layer)
model0a = tf_keras.models.Model(inputs=[input_layer], outputs=output_layer)
model0a.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mae'])

# Adding a callback to save only the best model
save_best_checkpoint = tf_keras.callbacks.ModelCheckpoint('model0a_best.h5', save_best_only=True)
model0a.fit(x_train, y_train,  epochs = 10, validation_data=(x_train__val, y_train__val), 
            callbacks=[save_best_checkpoint], verbose=0)

In [ ]:
# Adding a callback to Early Stop to avoid wasting time and resources
# with no further optimisation
stop_early_checkpoint = tf_keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Combine both callbacks. Use large epoch number because the model will stop when there 
# is no more better performance in the metrics
model0a.fit(x_train, y_train,  epochs = 100, validation_data=(x_train__val, y_train__val), 
            callbacks=[save_best_checkpoint, stop_early_checkpoint], 
            verbose=0)

### Visualisation using TensorBoard

In [ ]:
import os

In [ ]:
def get_run_logdir(root_logdir):
    import time
    run_id = time.strftime("r_%Y%m%d_%H%M%S")
    return os.path.join(root_logdir, run_id)

root_logdirp = os.path.join(os.curdir, "logs")
run_logdir = get_run_logdir(root_logdirp)
print(run_logdir)

In [ ]:
# Create the Tensorboard callback and use it
tensorboard_cb = tf_keras.callbacks.TensorBoard(run_logdir)
model0a.fit(x_train, y_train,  epochs = 100, validation_data=(x_train__val, y_train__val), 
            callbacks=[save_best_checkpoint, tensorboard_cb], 
            verbose=0)

Finally, you can access the TensorBoard with `python -m tensorboard.main --logdir=r_20200601_122625/`

<img src="img3c.png" width="750"/>

Additional Readings:

- (1)  https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html
- (2)  https://github.com/lutzroeder/Netron